In [94]:
from gtts import gTTS
import io
import pygame
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from pydantic import BaseModel
import pandas as pd
from langchain_core.messages import SystemMessage, HumanMessage
from google.cloud import bigquery
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from google.cloud import texttospeech
from google.oauth2 import service_account
import pygame
import time

# Authenticate with your service account
credentials = service_account.Credentials.from_service_account_file(
    r'C:\Users\Shubham\PycharmProjects\PythonProject\shubham-patil-gcp-vertexai json.json'
)

In [95]:
# load_dotenv()

In [96]:
from google.oauth2 import service_account
from google.cloud import aiplatform

import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r'C:\Users\Shubham\PycharmProjects\PythonProject\shubham-patil-gcp-vertexai json.json'


aiplatform.init(
    project="your-project-id",
    location="us-central1",
    credentials=credentials
)


In [97]:
from google.cloud import bigquery
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
   r'C:\Users\Shubham\PycharmProjects\PythonProject\shubham-patil-gcp-vertexai json.json'
)

# Create client with credentials
client = bigquery.Client(credentials=credentials, project=credentials.project_id)


In [98]:
# Define state:

class QueryState(TypedDict):
    que : str
    query : str
    result : pd.DataFrame
    answer : str

In [99]:
class llm_struct(BaseModel):
    query : str


# llm = ChatGroq(model="meta-llama/llama-4-maverick-17b-128e-instruct")

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
llm_struct_out = llm.with_structured_output(llm_struct)


In [100]:
# def query_generator():
#     que = 'Write a query to fetch top 3 salaried emplyoyees'
#     messages = [
#     SystemMessage(content=(
#         "You are an expert BigQuery SQL generator. "
#         "Convert the user’s natural language question into a valid BigQuery Standard SQL statement. "
#         "Return ONLY the SQL query inside the field 'query'."
#     )),
#     HumanMessage(content=que)
#     ]
#     query = llm_struct_out.invoke(messages).query
#     return print(query)
#
# query_generator()

In [101]:
# Define Nodes

def query_generator(que):
    schema_description = '''
    Table 1: customers
    Schema: customer_id STRING,
            first_name STRING,
            last_name STRING,
            email STRING,
            phone STRING,
            signup_date DATE,
            country STRING,
            is_active BOOLEAN

    Table 2: products
    Schema: product_id STRING,
            name STRING,
            description STRING,
            category STRING,
            price NUMERIC,
            stock_quantity INT64,
            supplier STRING,
            is_available BOOLEAN

    Table 3: orders
    Schema: order_id STRING,
            customer_id STRING,
            order_date DATE,
            shipping_address STRING,
            billing_address STRING,
            total_amount NUMERIC,
            status STRING,
            payment_method STRING

    Table 4: order_items
    Schema: order_item_id STRING,
            order_id STRING,
            product_id STRING,
            quantity INT64,
            unit_price NUMERIC,
            discount NUMERIC,
            tax NUMERIC,
            total_price NUMERIC
    '''

    messages = [
        SystemMessage(content=(
            "You are an expert BigQuery SQL generator. "
            "Convert the user’s natural language question into a valid BigQuery Standard SQL statement. "
            "Return ONLY the SQL query inside the field 'query'."
        )),
        HumanMessage(content=f"I have dataset `demo_1` with the following schema: {schema_description}\n\nNow, generate a query for: {que}")
    ]
    
    query = llm_struct_out.invoke(messages).query
    return {'query': query}


def query_runner(state:QueryState):
    query = state['query']
    #client = bigquery.Client(project="shubham-patil-gcp")
    query_job = client.query(query)
    df = query_job.result().to_dataframe()
    return {'result' : df}


def query_explainer(state:QueryState):
    result = state['result']
    que = state['que']

    # prompt = f'''explain given requirment {que} and fetched data from database{result}.\n
    #              do not explain structure of tables
    #              give short answer of requirement on the basis of fetched data
    #              if dataframe is empty then simple said there is no record for given requirement.
    #              strictly keep explaination withn 300 words'''

    prompt = prompt_template = f"""
                            You are a data analyst. 
                            I will give you:
                            1. The user’s original question.
                            2. The query result as a dataframe.

                            Your job:
                            - Provide a clear, human-friendly explanation of what the result means.
                            - Use simple and concise language (as if explaining to a business user).
                            - Highlight key insights, trends, or anomalies if present.
                            - If the dataframe is empty, politely explain that no records matched the query.
                            - Used EXTRACT(YEAR FROM date) for yearly comparison.
                            - If you need monthly comparison, use FORMAT_DATE('%Y-%m', date) instead.

                            Here is the information:

                            User Question:
                            {que}


                            Query Result (DataFrame):
                            {result}

                            Now explain the result in a human way."""




    ans = llm.invoke(prompt)
    return {'answer': ans}

In [102]:
prompt_template = """
You are a data analyst. 
I will give you:
1. The user’s original question.
2. The query result as a dataframe.

Your job:
- Provide a clear, human-friendly explanation of what the result means.
- Use simple and concise language (as if explaining to a business user).
- Highlight key insights, trends, or anomalies if present.
- If the dataframe is empty, politely explain that no records matched the query.
- Used EXTRACT(YEAR FROM date) for yearly comparison.
- If you need monthly comparison, use FORMAT_DATE('%Y-%m', date) instead.

Here is the information:

User Question:
{que}


Query Result (DataFrame):
{result}

Now explain the result in a human way.
"""


In [103]:
#Define Graph

graph = StateGraph(QueryState)

# Add Node

graph.add_node('query_generator',query_generator)
graph.add_node('query_runner', query_runner)
graph.add_node('query_explainer',query_explainer)

# Add Edges
graph.add_edge(START,'query_generator')
graph.add_edge('query_generator','query_runner')
graph.add_edge('query_runner','query_explainer')
graph.add_edge('query_explainer',END)
# graph.add_edge('query_runner',END)

# Compile Graph
app = graph.compile()

In [104]:
# app

In [105]:
initial_state = {'que' : 'Find customers whose total order value is in the top 10% percentile of all customers.'}


ans = app.invoke(initial_state)

# print(ans['query'].content)

C:\Users\Shubham\PycharmProjects\PythonProject\.venv\lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [106]:
print(ans['query'])#.content)

# print(ans['answer'].content)

# ans['result']

SELECT customer_id FROM ( SELECT customer_id, SUM(total_amount) AS total_order_value, PERCENT_RANK() OVER (ORDER BY SUM(total_amount) DESC) AS percentile FROM `demo_1.orders` GROUP BY customer_id ) WHERE percentile <= 0.1


In [107]:
# ans['query']

In [108]:
ans['result']

,customer_id
0,CUST009
1,CUST017
2,CUST099
3,CUST033
4,CUST090
5,CUST077
6,CUST043
7,CUST014
8,CUST035
9,CUST142


In [109]:
# from gtts import gTTS
# import os
#
# # Suppose this is your query result
# result = ans['answer'].content
#
# # Convert text to speech
# tts = gTTS(text=result, lang='en')
#
# # Save to an mp3 file
# tts.save("output.mp3")
#
# # Play the audio (works on Windows)
# os.system("start output.mp3")


In [110]:


# Your query result
# result = ans['answer'].content

# # Convert text to speech into memory (not saving to file)
# tts = gTTS(text=result, lang='en')
# fp = io.BytesIO()
# tts.write_to_fp(fp)
# fp.seek(0)
#
# # Initialize pygame mixer
# pygame.mixer.init()
# pygame.mixer.music.load(fp, 'mp3')
# pygame.mixer.music.play()
#
# # Keep program alive until playback is done
# while pygame.mixer.music.get_busy():
#     pygame.time.Clock().tick(10)
#
# # Free memory
# fp.close()
# del tts


In [111]:

client = texttospeech.TextToSpeechClient(credentials=credentials)

result = ans['answer'].content

# Set the text input to be synthesized
synthesis_input = texttospeech.SynthesisInput(text=result)

# Configure voice
voice = texttospeech.VoiceSelectionParams(
    language_code="en-US",
    ssml_gender=texttospeech.SsmlVoiceGender.FEMALE
)

# Configure audio
audio_config = texttospeech.AudioConfig(
    audio_encoding=texttospeech.AudioEncoding.MP3
)

# Synthesize speech
response = client.synthesize_speech(
    input=synthesis_input,
    voice=voice,
    audio_config=audio_config
)

# Save audio to file
file_path = "output.mp3"
with open(file_path, "wb") as out:
    out.write(response.audio_content)
    print(f'Audio content written to file "{file_path}"')

# Play audio using pygame
pygame.mixer.init()
pygame.mixer.music.load(file_path)
pygame.mixer.music.play()

# Wait until audio finishes playing
while pygame.mixer.music.get_busy():
    time.sleep(0.5)

# Delete the file after playing
pygame.mixer.music.unload()
os.remove(file_path)
print(f'File "{file_path}" deleted after playing')


Audio content written to file "output.mp3"
File "output.mp3" deleted after playing
